<a href="https://colab.research.google.com/github/thejuan27/github-slideshow/blob/main/sales_predictions_part_2_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
filename= '/content/sales_predictions.csv'
df = pd.read_csv(filename)
df.head()
df[df.columns[0]].count()


8523

In [2]:
#Checking the size of the dataset
print('Number of Rows',df.shape[0])
print('Number of Column',df.shape[1])

Number of Rows 8523
Number of Column 12


In [3]:
#Check the data type of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
#Removing Dupicates
df.drop_duplicates().any()
#There are no duplicates

bool_series = df.duplicated(keep='last')
bool_series

0       False
1       False
2       False
3       False
4       False
        ...  
8518    False
8519    False
8520    False
8521    False
8522    False
Length: 8523, dtype: bool

In [5]:
#Determine the missing values
df.isnull().sum()      

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [6]:
#For the missing value under Item_Weight, I use interpolation to determine the missing value, this will help 
#to determine the average weight of the item

df['Item_Weight'] = df.loc[0:,'Item_Weight'].interpolate(method = 'linear')
#For outlet size, I use a constant to fill the null values. this way, I can still use the data but it will be subdivided to its categories 
#by not specified and other category
df['Outlet_Size'] = df.loc[0:,'Item_Weight'].fillna(0)
df.isnull().sum()    

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [7]:
#Checking inconsistent values in categories

#Check first the Categorical Value of the Dataset
df_cat = df.select_dtypes(include = ['object'])
cat_names = df_cat.columns

for column in cat_names:
  print(column,df[column].unique())

#Upon Check, Column for Item_Fat_Content and Outlet_Type have inconsistencies in their data

Item_Identifier ['FDA15' 'DRC01' 'FDN15' ... 'NCF55' 'NCW30' 'NCW05']
Item_Fat_Content ['Low Fat' 'Regular' 'low fat' 'LF' 'reg']
Item_Type ['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']
Outlet_Identifier ['OUT049' 'OUT018' 'OUT010' 'OUT013' 'OUT027' 'OUT045' 'OUT017' 'OUT046'
 'OUT035' 'OUT019']
Outlet_Location_Type ['Tier 1' 'Tier 3' 'Tier 2']
Outlet_Type ['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']


In [8]:
#Checking inconsistent values in number types

df_num = df.select_dtypes(include = ['int64','float64'])
num_names = df_num.columns

for column in num_names:
  print(column,df[column].unique())

#Check for the minimum and maximum value
  if column == 'Item_Weight':
    print('Max',df['Item_Weight'].max())
    print('Min',df['Item_Weight'].min())
  if column == 'Item_Visibility':
    print('Max',df['Item_Visibility'].max())
    print('Min',df['Item_Visibility'].min()) 
  if column == 'Item_MRP':
    print('Max',df['Item_MRP'].max())
    print('Min',df['Item_MRP'].min())
  if column == 'Outlet_Establishment_Year':
    print('Max',df['Outlet_Establishment_Year'].max())
    print('Min',df['Outlet_Establishment_Year'].min())
  if column == 'Outlet_Size ':
    print('Max',df['Outlet_Size '].max())
    print('Min',df['Outlet_Size '].min())
  if column == 'Item_Outlet_Sales':
    print('Max',df['Item_Outlet_Sales'].max())
    print('Min',df['Item_Outlet_Sales'].min())

#Some values are subjective. Like the outlet sales

Item_Weight [ 9.3         5.92       17.5        ... 19.11666667 18.23333333
 13.8725    ]
Max 21.35
Min 4.555
Item_Visibility [0.0160473  0.01927822 0.01676007 ... 0.03518627 0.14522065 0.04487828]
Max 0.328390948
Min 0.0
Item_MRP [249.8092  48.2692 141.618  ...  57.5904 214.5218  75.467 ]
Max 266.8884
Min 31.29
Outlet_Establishment_Year [1999 2009 1998 1987 1985 2002 2007 1997 2004]
Max 2009
Min 1985
Outlet_Size [ 9.3         5.92       17.5        ... 19.11666667 18.23333333
 13.8725    ]
Item_Outlet_Sales [3735.138   443.4228 2097.27   ... 7182.6504 3608.636  2778.3834]
Max 13086.9648
Min 33.29


In [9]:
#Cleaning the values under Item_Fat_Content
df['Item_Fat_Content'].value_counts()


Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [10]:
#Changing LF to Low Fat by replace
df['Item_Fat_Content'].replace('LF', 'Low Fat',inplace=True)
df['Item_Fat_Content'].replace('low fat', 'Low Fat',inplace=True)
df['Item_Fat_Content'].replace('reg', 'Regular',inplace=True)

In [12]:
#Cleaning the values under Outlet_Type
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [15]:
#Will replace the Grocery Store to Supermarket Type1 because it has the largest count, which could be the basic store
df['Outlet_Type'].replace('Grocery Store', 'Supermarket Type1',inplace=True)
df['Outlet_Type'].value_counts()

Supermarket Type1    6660
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [ ]:
#Getting the min, max and mean of the Item_Visibility Column

print('The Maximum value for the column Item_Visibility is',df['Item_Visibility'].max())
print('The Minimum value for the column Item_Visibility is',df['Item_Visibility'].min())
print('The Mean value for the column Item_Visibility is',df['Item_Visibility'].mean())


In [22]:
#Getting the min, max and mean of the Item_Weight Column

print('The Maximum value for the column Item_Weight',df['Item_Weight'].max())
print('The Minimum value for the column Item_Weight is',df['Item_Weight'].min())
print('The Mean value for the column Item_Weight is',df['Item_Weight'].mean())

The Maximum value for the column Item_Weight 21.35
The Minimum value for the column Item_Weight is 4.555
The Mean value for the column Item_Weight is 12.831222280887394


In [23]:
#Getting the min, max and mean of the Item_MRP Column

print('The Maximum value for the column Item_MRP',df['Item_MRP'].max())
print('The Minimum value for the column Item_MRPt is',df['Item_MRP'].min())
print('The Mean value for the column Item_MRP is',df['Item_MRP'].mean())

The Maximum value for the column Item_MRP 266.8884
The Minimum value for the column Item_MRPt is 31.29
The Mean value for the column Item_MRP is 140.9927819781768


In [24]:
#Getting the min, max and mean of the Outlet_Establishment_Year Column

print('The Maximum value for the column Outlet_Establishment_Year',df['Outlet_Establishment_Year'].max())
print('The Minimum value for the column Outlet_Establishment_Year is',df['Outlet_Establishment_Year'].min())
print('The Mean value for the column Outlet_Establishment_Year',df['Outlet_Establishment_Year'].mean())

The Maximum value for the column Outlet_Establishment_Year 2009
The Minimum value for the column Outlet_Establishment_Year is 1985
The Mean value for the column Outlet_Establishment_Year 1997.8318667135984


In [25]:
#Getting the min, max and mean of the Outlet_Size Column
print('The Maximum value for the column Outlet_Size',df['Outlet_Size'].max())
print('The Minimum value for the column Outlet_Size',df['Outlet_Size'].min())
print('The Mean value for the column Outlet_Size',df['Outlet_Size'].mean())

The Maximum value for the column Outlet_Size 21.35
The Minimum value for the column Outlet_Size 4.555
The Mean value for the column Outlet_Size 12.831222280887394


In [26]:
#Getting the min, max and mean of the Item_Outlet_Sales Column
print('The Maximum value for the column Item_Outlet_Sales',df['Item_Outlet_Sales'].max())
print('The Minimum value for the column Item_Outlet_Sales',df['Item_Outlet_Sales'].min())
print('The Mean value for the column Item_Outlet_Sales',df['Item_Outlet_Sales'].mean())

The Maximum value for the column Item_Outlet_Sales 13086.9648
The Minimum value for the column Item_Outlet_Sales 33.29
The Mean value for the column Item_Outlet_Sales 2181.2889135750365
